In [2]:
 
import numpy as np 
import pandas as pd 
from collections import defaultdict
import pandas_summary as ps
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate, KFold, GridSearchCV
from surprise import SVD, NMF, SlopeOne, KNNBasic, KNNWithMeans, KNNBaseline, CoClustering, BaselineOnly, NormalPredictor, KNNWithZScore
from surprise.model_selection import cross_validate, train_test_split, split, GridSearchCV, LeaveOneOut, KFold
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
import datetime
from operator import itemgetter
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.matrix_factorization import SVDpp, SVD


In [1]:
import os
for dirname, _, filenames in os.walk('../data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../data/anime.csv
../data/rating.csv


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
folder = '../data/'
anime_df = pd.read_csv(folder+'anime.csv')
rate_df = pd.read_csv(folder+'rating.csv')
print('anime: ', anime_df.shape)
print('rating: ', rate_df.shape)
anime_df.head()

anime:  (12294, 7)
rating:  (7813737, 3)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
full_df = rate_df.merge(anime_df, how='left', left_on=['anime_id'], right_on=['anime_id'])
df = full_df[full_df['rating_x'] != -1]
df.head()

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
47,1,8074,10,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892.0
81,1,11617,10,High School DxD,"Comedy, Demons, Ecchi, Harem, Romance, School",TV,12,7.70,398660.0
83,1,11757,10,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100.0
101,1,15451,10,High School DxD New,"Action, Comedy, Demons, Ecchi, Harem, Romance,...",TV,12,7.87,266657.0
153,2,11771,10,Kuroko no Basket,"Comedy, School, Shounen, Sports",TV,25,8.46,338315.0


In [82]:
dfs = ps.DataFrameSummary(full_df)
print('categoricals: ', dfs.categoricals.tolist())
print('numerics: ', dfs.numerics.tolist())
dfs.summary()

categoricals:  ['name', 'genre', 'type', 'episodes']
numerics:  ['user_id', 'anime_id', 'rating_x', 'rating_y', 'members']


,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
count,1026768.0,1026768.0,1026768.0,NaN,NaN,NaN,NaN,1026768.0,1026768.0
mean,4921.428382,8816.599805,6.163859,NaN,NaN,NaN,NaN,7.652481,177928.978117
std,2674.891672,8987.894924,3.700448,NaN,NaN,NaN,NaN,0.670284,188823.002756
min,1.0,1.0,-1.0,NaN,NaN,NaN,NaN,2.0,43.0
25%,2665.0,1130.0,6.0,NaN,NaN,NaN,NaN,7.26,43444.0
50%,5189.0,5681.0,7.0,NaN,NaN,NaN,NaN,7.68,108170.0
75%,7174.0,14227.0,9.0,NaN,NaN,NaN,NaN,8.13,244193.0
max,9786.0,34325.0,10.0,NaN,NaN,NaN,NaN,9.37,1013917.0
counts,1026768,1026768,1026768,1026768,1026754,1026768,1026768,1026768,1026768
uniques,9786,8335,11,8335,2867,6,161,512,6360


In [83]:
min_anime_ratings = 250
filter_anime = df['anime_id'].value_counts() > min_anime_ratings
filter_anime = filter_anime[filter_anime].index.tolist()

min_user_ratings = 250
filter_users = df['user_id'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['anime_id'].isin(filter_anime)) & (df['user_id'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))
df_new

The original data frame shape:	(836438, 9)
The new data frame shape:	(200878, 9)


,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
302,5,6,8,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069
303,5,15,6,Eyeshield 21,"Action, Comedy, Shounen, Sports",TV,145,8.08,83648
306,5,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
307,5,22,5,Prince of Tennis,"Action, Comedy, School, Shounen, Sports",TV,178,8.04,87643
308,5,24,1,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553
...,...,...,...,...,...,...,...,...,...
1025199,9754,31478,9,Bungou Stray Dogs,"Action, Comedy, Mystery, Seinen, Supernatural",TV,12,7.76,187805
1025201,9754,31580,7,Ajin,"Action, Horror, Mystery, Seinen, Supernatural",TV,13,7.78,160972
1025202,9754,31637,9,"Gate: Jieitai Kanochi nite, Kaku Tatakaeri 2nd...","Action, Adventure, Fantasy, Military",TV,12,7.97,153501
1025206,9754,31798,8,Kiznaiver,"Drama, Sci-Fi",TV,12,7.67,197912


In [84]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_new[['user_id', 'anime_id', 'rating_x']], reader)
data

In [85]:
df_new[['user_id', 'anime_id', 'rating_x']]

,user_id,anime_id,rating_x
302,5,6,8
303,5,15,6
306,5,20,6
307,5,22,5
308,5,24,1
...,...,...,...
1025199,9754,31478,9
1025201,9754,31580,7
1025202,9754,31637,9
1025206,9754,31798,8


In [86]:
algo = SVD(biased=False)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)

{'test_rmse': array([1.12849712, 1.13658537, 1.12208376, 1.14282595, 1.13181246]),
 'fit_time': (4.03079628944397,
  2.856555461883545,
  2.5965774059295654,
  2.7733566761016846,
  3.5584263801574707),
 'test_time': (0.4236314296722412,
  0.25617504119873047,
  0.28031158447265625,
  0.44156551361083984,
  0.275043249130249)}

In [87]:
def get_top_n(predictions, n=10, minV = 0):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        top_n[uid] = [y for y in top_n[uid] if y[1] >= minV]

    return top_n

def HitRate(topNPredicted, leftOutPredictions):
  hits = 0
  total = 0

  # For each left-out rating
  for leftOut in leftOutPredictions:

    userID = leftOut[0]
    leftOutMovieID = leftOut[1]
    # Is it in the predicted top n for this user?
    hit = False

    for movieID, predictedRating in topNPredicted[userID]:
      if leftOutMovieID == movieID:
        hit = True
        break
    if (hit) :
      hits += 1

    total += 1

  # Compute overall precision
  print("hits: ", hits, "total : ", total)
  return hits/total


def AverageReciprocalHitRank(topNPredicted, leftOutPredictions):
    summ = 0
    total = 0

  # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        # Is it in the predicted top n for this user?
        hitRank = 0
        rank = 0

        for movieID, predictedRating in topNPredicted[userID]:
            rank = rank + 1
            if leftOutMovieID == movieID:
                hitRank = rank
                break
        if (hitRank>0) :
          summ += 1.0/hitRank

        total += 1
    return summ / total


def CumulativeHitRate(topNPredicted, leftOutPredictions, cutoff):
    hits = 0
    total = 0

  # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        if (leftOut[2] >= cutoff):
            # Is it in the predicted top n for this user?
            hit = False

            for movieID, predictedRating in topNPredicted[userID]:
                if leftOutMovieID == movieID:
                    hit = True
                    break
            if (hit) :
              hits += 1

            total += 1
            # Compute overall precision
    return hits/total

def RatingHitRate(topNPredicted, leftOutPredictions):
    hits = defaultdict(float)
    total = defaultdict(float)

  # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
    # Is it in the predicted top n for this user?
        hit = False

        for movieID, predictedRating in topNPredicted[userID]:
          if leftOutMovieID == movieID:
            hit = True
            break
        if (hit) :
            hits[leftOut[2]] += 1
        total[leftOut[2]] += 1
    for rating in sorted(hits.keys()):
        print(rating, hits[rating] / total[rating])


def user_coverage(topNPredicted, minRating):
    user_coverage = 0
    for userPrediction in topNPredicted.items():
        for movieRating in userPrediction[1]:
            if movieRating[1]>minRating:
                user_coverage+=1
                break
    print("user coverage ", user_coverage/len(topNPredicted))


def get_hitrate_results(algo, train_loocv, test_loocv, topN, minVal):
  algo.fit(train_loocv)
  left_out_predictions = algo.test(test_loocv)
  loocv_anti_testset = train_loocv.build_anti_testset()
  all_predictions = algo.test(loocv_anti_testset)
  top_n_predicted = get_top_n(all_predictions, topN, minVal)
  RatingHitRate(top_n_predicted, left_out_predictions)
  hitrate = HitRate(top_n_predicted, left_out_predictions)

  accuracy.mae(left_out_predictions)
  accuracy.rmse(left_out_predictions)

  print(f'HitRate: {hitrate}')
  hitrate = CumulativeHitRate(top_n_predicted, left_out_predictions, 4)
  print(f'HitRateCumul: {hitrate}')
  print("Average Reciprocal Hit Rank: ", AverageReciprocalHitRank(top_n_predicted, left_out_predictions))
  user_coverage(top_n_predicted, 4)
  return all_predictions



def hyperTune(param_grid, data, modelTarget, targetedMetric, cv = KFold(n_splits=5, random_state=777)):
    print("starting hyper tune for "+ str(modelTarget), "with" ,param_grid )
    gs = GridSearchCV(modelTarget, param_grid, measures=["rmse","mae"], cv=cv)
    gs.fit(data)
    print("RMSE score", gs.best_score['rmse'], " with ", gs.best_params)
    print("mae score", gs.best_score['mae'], " with ", gs.best_params)

    return gs.best_score[targetedMetric], gs.best_params[targetedMetric], modelTarget


In [88]:
cv = KFold(n_splits=5, random_state=777)
cvSlope = KFold(n_splits=2, random_state=777)
targets = [
           {"name": "Co-clustering", "param_grid": {"n_epochs": [20, 30], "n_cltr_u": [3, 6], "n_cltr_i": [3, 6]},"verbose": [False], "model": CoClustering, "CV": cv},
           {"name":"Slope one", "param_grid": {}, "verbose": [False], "model": SlopeOne, "CV":cvSlope},
           {"name":"knn basic","param_grid":{"sim_options": {"name": ["msd"], "min_support":[2,5,10]}, "k": [30,40], "min_k": [1,10], "verbose": [False], "user_based": [True, False]},"model":KNNBasic, "CV":cv},
           {"name":"knn means","param_grid":{"sim_options": {"name": ["msd"], "min_support":[2,5,10]}, "k": [30,40], "min_k": [1,10], "verbose": [False], "user_based": [True, False]},"model":KNNWithMeans, "CV":cv},
           {"name":"knn Zscore","param_grid":{"sim_options": {"name": ["msd"], "min_support":[2,5,10]}, "k": [30,40], "min_k": [1,10], "verbose": [False], "user_based": [True, False]},"model":KNNWithZScore, "CV":cv},
           {"name":"Random Base", "param_grid":{}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Baseline ALS", "param_grid":{"bsl_options": {"n_epochs": [5, 10], "reg_u": [5, 10, 20], "reg_i": [5, 10, 20], "method": ["als"]}}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Baseline SGD", "param_grid":{"bsl_options": {"n_epochs": [5, 10],  "learning_rate": [0.005, 0.0005], "reg": [0.02, 0.01], "method": ["sgd"]}}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Matrix factorization based", "param_grid":{"lr_all":[0.007, 0.01, 0.15]}, "verbose": [False], "model":SVD, "CV":cv},
           {"name":"Matrix factorization based + implicit rating", "param_grid":{"lr_all":[0.007, 0.01, 0.15]}, "verbose": [False], "model":SVDpp, "CV":cv}
]
# knns.KNNWithMeans
targetedMetric = "mae"
bestScoringModel = [np.inf, None, None]


LOOCV = LeaveOneOut(n_splits=1, random_state=777)
train_loocv, test_loocv  = LOOCV.split(data).__next__()
for target in targets:
    print("****** doing ", target)
    np.random.seed(40)
    scoringModel, modelParam, model = hyperTune( target["param_grid"], data, target["model"], targetedMetric, target["CV"])
    print("best mae for ", target["name"], ":", scoringModel, " with parameters :", target["param_grid"])
    if scoringModel < bestScoringModel[0]:
        print("found new best tuned model !")
        bestScoringModel[0] = scoringModel
        bestScoringModel[1] = modelParam
        bestScoringModel[2] = model

    get_hitrate_results(bestScoringModel[2](), train_loocv, test_loocv, 10, 2)
    print("************************")


get_hitrate_results(bestScoringModel[2](), train_loocv, test_loocv, 10, 2)

In [89]:
def get_user_top_n(predictions, id, n=10):
 
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
      if(uid == id):
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    

    return top_n

In [219]:
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_user_top_n(predictions, 5, n=10)
if(len(top_n) == 0):
        print("Utilisateur introuvable")
else:
  result_recommendation =[]
  for uid, user_ratings in top_n.items():
      # print(uid, [iid for (iid, _) in user_ratings])
      result_recommendation = uid, [iid for (iid, _) in user_ratings]
      recommendation_df = pd.DataFrame({
        "anime_id" : [],
        "name" : []
     })
      
  for anime_id in result_recommendation[1]:
    recommendation_df.loc[len(recommendation_df.index)] = [anime_id, anime_df.loc[anime_df['anime_id'] == anime_id]["name"].values[0]]
  print("Resultats des",len(recommendation_df.index),"meilleurs recommendations pour le user_id",result_recommendation[0])
  print(recommendation_df)
 

Resultats des 10 meilleurs recommendations pour le user_id  5
   anime_id                              name
0       416                   Kurenai no Buta
1         1                      Cowboy Bebop
2       227                              FLCL
3     16001      Kokoro Connect: Michi Random
4       578                    Hotaru no Haka
5         5   Cowboy Bebop: Tengoku no Tobira
6      5114  Fullmetal Alchemist: Brotherhood
7       523                  Tonari no Totoro
8        33                           Berserk
9       572          Kaze no Tani no Nausicaä
